In [1]:
from config.settings import SRC_DIR
import os
os.chdir(SRC_DIR)

In [2]:
from document_processing_system.components.data_ingestion import DataIngestion
from document_processing_system.components.data_extraction import DataExtraction

pdf = 'PO166939-204865'

data_ingester = DataIngestion(f'qc_data/qc_templates/{pdf}.pdf')
data_extracter = DataExtraction(data_ingester)
pages, tables, images = data_extracter.extract_data()
all_items = data_extracter.filter_pages(pages)
data_extracter.display_data(all_items)

Extracting pages:   0%|          | 0/15 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/15 [00:00<?, ?pages/s]


--------Page: 2-----------
Intake Pallet QC Inspection Report
Customer DPS PO 166939 / ISS PO 204865
Supplier Code : GUIM€
Supplier : Guimera Fruits
COO : Spain
Vehicle No : Received : 03/09/2024 09:59:25
Vessel : Inspection Date : 04/09/2024 06:35:59 Print date : 04/09/2024
Apricots 20x320g Punnet
ISS Pallet ID : 7761770 Freshness Technology :
Supplier Pallet ID : 5808239 Punnet / Pad Type : N/A /
Customer Pallet ID : 5808239 Outer :
Variety : Fardao Brand : CORE
Grower : / Organic? : NO Does Pallet Meet Spec? : YES
GGN : / PLU? :
BLUE Expected Qty 80
Orchard/Farm : End Customer : Tesco
Received Qty 80
Harvest Date : 30/08/2024 DP : 7
Total Defects : 5.00%
Size/Calibre : 40/45 Packhouse :
Estimated Yield : 100%
Lot Number : 508164 Inspector : Hanna.Dziuba
Minor : Dry Splits: 1.67%
Minor : Puncture: 1.67%
Major : Scarring: 1.67%
Defects Tot : 0.00% Defects Fruit Total : 0 Packs With Defects : 0.00%
Waste Tot : 0.00% Waste Fruit Total : 0 Packs With Waste : 0%
Minor Defects Tot : 3.33%

In [3]:
import os
from document_processing_system.components.model_inference import ModelInference
from config.settings import DATA_DIR

# Define the PDF folder path
pdf_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'qc_reports_14b')

# Ensure the directory exists
os.makedirs(pdf_folder, exist_ok=True)

# Initialize the model
model = ModelInference(model_name="Qwen/Qwen2.5-14B")

INFO:root:Loading model and tokenizer...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0
INFO:root:Model loaded successfully!


In [4]:
# print(model.get_prompt(all_items[0]))
# Process items
response = model.process_items(all_items, output_folder=pdf_folder)

Processing Items:   0%|          | 0/10 [00:00<?, ?item/s]

INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_0.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_1.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_2.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_3.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_4.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_5.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_reports_14b\qc_report_item_6.json
INFO:root:Saved: C:\Users\ai.users\local_ocr\data\qc_data\qc_reports_output\PO166939-204865\qc_re

In [5]:
response

[[{'product_name': 'Apricots 20x320g Punnet',
   'RAG': 'BLUE',
   'expected_qty': '80',
   'received_qty': '80',
   'supplier_code': 'GUIM€',
   'supplier': 'Guimera Fruits',
   'coo': 'Spain',
   'received_date': '2024-09-03 09:59:25',
   'inspection_date': '2024-09-04 06:35:59',
   'print_date': '2024-09-04',
   'iss_pallet_id': 7761770,
   'supplier_pallet_id': 5808239,
   'customer_pallet_id': 5808239,
   'variety': 'Fardao',
   'brand': 'CORE',
   'organic': 'NO',
   'does_pallet_meet_spec': 'YES',
   'end_customer': 'Tesco',
   'harvest_date': '2024-08-30',
   'dp': 7,
   'total_defects_percentage': 5.0,
   'size_calibre': '40/45',
   'lot_number': 508164,
   'inspector': 'Hanna.Dziuba',
   'estimated_yield_percentage': 100.0,
   'defects_fruit_total': 0,
   'packs_with_defects': 0,
   'waste_tot': 0,
   'waste_fruit_total': 0,
   'packs_with_waste': 0,
   'minor_defects_tot': 3.33,
   'minor_fruit_total': 2,
   'major_defects_tot': 1.67,
   'major_fruit_total': 1,
   'packs_wit

In [6]:
import polars as pl
flat_data = [item for sublist in response for item in sublist]

data = pl.DataFrame(flat_data)
data

product_name,RAG,expected_qty,received_qty,supplier_code,supplier,coo,received_date,inspection_date,print_date,iss_pallet_id,supplier_pallet_id,customer_pallet_id,variety,brand,organic,does_pallet_meet_spec,end_customer,harvest_date,dp,total_defects_percentage,size_calibre,lot_number,inspector,estimated_yield_percentage,defects_fruit_total,packs_with_defects,waste_tot,waste_fruit_total,packs_with_waste,minor_defects_tot,minor_fruit_total,major_defects_tot,major_fruit_total,packs_with_major,box_pack_weights,under_weight_percentage,weight_readings,fruit_weights,under_size_percentage,qa_comments,packs_fruits_inspected_sample_size,boxes_inspected
str,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,f64,str,i64,str,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,i64,str,f64,str,str,f64,str,i64,i64
"""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7761770,5808239,5808239,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 340g ; Min 326g; Max 367g""",0.0,"""326 326 328 330 330 331 332 33…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60,0
"""Apricots 20x320g Punnet""","""AMBER""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7761771,5808238,5808238,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,10.83,"""40/45""",508165,"""Hanna.Dziuba""",94.0,0,0,2.5,3,0,5.0,6,3.33,4,0,"""Avg 352g ; Min 335g; Max 375g""",0.0,"""335 337 338 338 340 341 341 34…","""Avg 44g 44""",0.0,"""Waste. Bruising. Dry splits. S…",120,0
"""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763099,7763099,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 345g ; Min 324g; Max 361g""",0.0,"""324 330 331 335 337 338 341 34…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60,0
"""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763100,7763100,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 348g ; Min 330g; Max 380g""",0.0,"""330 331 332 333 336 340 341 34…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60,0
"""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763101,7763101,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 341g ; Min 324g; Max 359g""",0.0,"""324 332 333 336 336 337 337 33…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60,0
"""Apricots 20x320g Punnet""","""AMBER""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763102,7763102,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,10.83,"""35/40""",508164,"""Hanna.Dziuba""",94.0,0,0,2.5,3,0,5.0,6,3.33,4,0,"""Avg 350g ; Min 327g; Max 379g""",0.0,"""327 335 339 339 340 341 341 34…","""Avg 44g 44""",0.0,"""Waste. Bruising. Dry splits. S…",120,0
"""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763103,7763103,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""

In [8]:
excel_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'excel')

# Save data

In [15]:
data = data.with_columns(pl.lit(pdf, pl.String).alias("pdf_id")).select(["pdf_id"] + data.columns[:-1])
data.write_excel(f"{excel_folder}/data.xlsx")
data

pdf_id,product_name,RAG,expected_qty,received_qty,supplier_code,supplier,coo,received_date,inspection_date,print_date,iss_pallet_id,supplier_pallet_id,customer_pallet_id,variety,brand,organic,does_pallet_meet_spec,end_customer,harvest_date,dp,total_defects_percentage,size_calibre,lot_number,inspector,estimated_yield_percentage,defects_fruit_total,packs_with_defects,waste_tot,waste_fruit_total,packs_with_waste,minor_defects_tot,minor_fruit_total,major_defects_tot,major_fruit_total,packs_with_major,box_pack_weights,under_weight_percentage,weight_readings,fruit_weights,under_size_percentage,qa_comments,packs_fruits_inspected_sample_size
str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,f64,str,i64,str,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,i64,str,f64,str,str,f64,str,i64
"""PO166939-204865""","""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7761770,5808239,5808239,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 340g ; Min 326g; Max 367g""",0.0,"""326 326 328 330 330 331 332 33…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60
"""PO166939-204865""","""Apricots 20x320g Punnet""","""AMBER""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7761771,5808238,5808238,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,10.83,"""40/45""",508165,"""Hanna.Dziuba""",94.0,0,0,2.5,3,0,5.0,6,3.33,4,0,"""Avg 352g ; Min 335g; Max 375g""",0.0,"""335 337 338 338 340 341 341 34…","""Avg 44g 44""",0.0,"""Waste. Bruising. Dry splits. S…",120
"""PO166939-204865""","""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763099,7763099,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 345g ; Min 324g; Max 361g""",0.0,"""324 330 331 335 337 338 341 34…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60
"""PO166939-204865""","""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763100,7763100,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 348g ; Min 330g; Max 380g""",0.0,"""330 331 332 333 336 340 341 34…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60
"""PO166939-204865""","""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763101,7763101,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,5.0,"""40/45""",508164,"""Hanna.Dziuba""",100.0,0,0,0.0,0,0,3.33,2,1.67,1,0,"""Avg 341g ; Min 324g; Max 359g""",0.0,"""324 332 333 336 336 337 337 33…","""Avg 44g 44""",0.0,"""Dry splits. Scarring. Isolated…",60
"""PO166939-204865""","""Apricots 20x320g Punnet""","""AMBER""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-04 06:35:59""","""2024-09-04""",7763102,7763102,null,"""Fardao""","""CORE""","""NO""","""YES""","""Tesco""","""2024-08-30""",7,10.83,"""35/40""",508164,"""Hanna.Dziuba""",94.0,0,0,2.5,3,0,5.0,6,3.33,4,0,"""Avg 350g ; Min 327g; Max 379g""",0.0,"""327 335 339 339 340 341 341 34…","""Avg 44g 44""",0.0,"""Waste. Bruising. Dry splits. S…",120
"""PO166939-204865""","""Apricots 20x320g Punnet""","""BLUE""","""80""","""80""","""GUIM€""","""Guimera Fruits""","""Spain""","""2024-09-03 09:59:25""","""2024-09-0